In [1]:
# change file name in line 29, text must be in column 3.
# output file is called limit_post.csv. Change this name before you run the script each time.
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import pandas as pd
import csv
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer


In [2]:
def isNum(passedString):
    return any(i.isdigit() for i in passedString)

posts_found_with_brand = []
limited_post = []
limited_post2 = []
final_list = []
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
newStopWords = ["n't","'s","'m","also","'d","'ve","could","would","ca"]
removeStopWords = ['above','below','up','down']
stopwords.extend(newStopWords)
for word in removeStopWords:
    stopwords.remove(word)
inter1 = []
completeComment_list = []


[nltk_data] Downloading package stopwords to /Users/harsh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
president = pd.read_pickle('president_df')
modi = pd.read_excel('modi.xlsx')

inter1 = []
for i in modi['Speech']:
    inter1.append(i)
for i in president['Speech']:
    inter1.append(i)
    
len(inter1)

In [8]:
for f in inter1:
    stop_words = set(stopwords)
    sentence_list = sent_tokenize(f)
    temp_list = []
    for sentence in sentence_list:
        word_tokens = word_tokenize(sentence)
        wordLower_tokens = []
        wordLower_tokens = [g.lower() for g in word_tokens]
        firstF_list = [h for h in wordLower_tokens if not h in stop_words]
        firstF_list = []
        secondF_list = []
        for i in wordLower_tokens:
            if i not in stop_words:
                firstF_list.append(i)
        for j in firstF_list:
            if(isNum(j) == False):
                wordNoPunctuation = re.sub('[\W_]+', '', j)
                secondF_list.append(wordNoPunctuation)
            else:
                secondF_list.append(j)
        temp_list.append(secondF_list)
    completeComment_list.append(temp_list)
posts_found_with_brand = completeComment_list



In [33]:
#word list
topic1words = ['peace','war','country','security','freedom']
topic2words = ['health','tax','economy','education','reform']
topic3words = ['order','development','action','trade']
topic4words = ['virus','job','money','administration','election']

topic_words = [['peace','war','country','security','freedom'],['health','tax','economy','education','reform'],['order','development','action','trade'],['virus','job','money','administration','election']]

In [30]:
def get_sentiment(rating_data):
    sid = SentimentIntensityAnalyzer()

    # these comments are some common operations used to change the vader lexicon

    #sid.lexicon.clear()
    #newWords = {'positive': 1.0, 'negative': -1.0}
    #sid.lexicon.update(newWords)

    #score = sid.polarity_scores(sentence)
    #compound = score.get('compound')
    rating_data['sent_neg'] = -10
    rating_data['sent_neu'] = -10
    rating_data['sent_pos'] = -10
    rating_data['sent_compound'] = -10
    for i in range(len(rating_data)):
        sentence = rating_data['Sentences'][i]
        ss = sid.polarity_scores(sentence)
        rating_data.iloc[i, 1] = float(ss['neg'])
        rating_data.iloc[i, 2] = ss['neu']
        rating_data.iloc[i, 3] = ss['pos']
        rating_data.iloc[i, 4] = ss['compound']
    return rating_data


In [35]:
def find_sentiment(topic_words):
    limited_post =[]
    for post in posts_found_with_brand:
        sentence_with_attribute = []
        position_in_sentences = []
        for sentence in post:
            if '' in sentence:
                sentence.remove('')
                #might have to run seperately for all topics
            for target in topic_words:
                sentence= ['abcdefgh' if value==target else value for value in sentence]
            position = 0
            for i in sentence:
                if(i=='abcdefgh'):
                    position_in_sentences.append(position)
                    sentence_with_attribute.append(sentence)
                position = position + 1
        j = 0

        for sentence in sentence_with_attribute:
            limited_sentence = []
            for i in range(len(sentence)):
                if(abs(i - position_in_sentences[j]) < 3):
                    limited_sentence.append(sentence[i])
            j=j+1
            limited_post.append(limited_sentence)
    for sent in limited_post:
        limited_post2.append(' '.join(sent))

    df = pd.DataFrame()
    df['Sentences']  = limited_post2
    
    rating_data=df
    # rating_data = pd.read_csv("limit_post.csv")
    rating_data = rating_data.rename(columns={ rating_data.columns[0]: "Sentences" })

    sentiment_data = get_sentiment(rating_data)
    return(sentiment_data)

In [37]:
names = ['topic1_sentiment.csv','topic2_sentiment.csv','topic3_sentiment.csv','topic4_sentiment.csv']
for i in range(0, len(topic_words)):
    result = find_sentiment(topic_words[i])
    result.to_csv(names[i])
    
    
    